Contribution of each group member:
* 1) Bhavya Kanzariya (AI21MTECH11002) : Extractive Summarizer, Evaluation and Literature Survey
* 2) Kashyap Parmar (AI21MTECH14006) : Conversion of Video to Text and Literature Survey
* 3) Darshan Dobariya (AI21MTECH14008) : Abstractive Summarizer and Literature Survey
* 4) Soham Bhatt (SM21MTECH14004) : End-to-End product implementation with Flask framework and Literature Survey
* 5) Yogesh Ahirwar (CS21MTECH11015) : Literature Survey and LSA implementation.

## Import dependencies 

In [ ]:
import pytube
import pandas as pd
import moviepy.editor as mp
import math
import os
import speech_recognition as sr
from pydub import AudioSegment
from pydub.utils import make_chunks 
# Importing dependencies from transformers
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from rouge_score import rouge_scorer

## Downloading Video

In [18]:
link = "https://www.youtube.com/watch?v=MnT1xgZgkpk&t=4s"
yt = pytube.YouTube(link)
path = "C:\\Users\\dobar\\Desktop\\2nd Sem\\Deep Learning\\Project\\Final\\"
yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').first().download(path)

'C:\\Users\\dobar\\Desktop\\2nd Sem\\Deep Learning\\Project\\Final\\What happens when our computers get smarter than we are  Nick Bostrom.mp4'

In [19]:
clip = mp.VideoFileClip("C:\\Users\\dobar\\Desktop\\2nd Sem\\Deep Learning\\Project\\Final\\ex.mp4")
clip.audio.write_audiofile("C:\\Users\\dobar\\Desktop\\2nd Sem\\Deep Learning\\Project\\Final\\test.wav",codec='pcm_s16le')

MoviePy - Writing audio in C:\Users\dobar\Desktop\2nd Sem\Deep Learning\Project\Final\test.wav


MoviePy - Done.


## Getting Final Text from Video

In [20]:
w=[]
def get_large_audio_transcription(path):
    r = sr.Recognizer()

    sound = AudioSegment.from_wav(path)  

    myaudio = AudioSegment.from_file(path, "wav") 
    chunk_length_ms = 20*1000 
    chunks = make_chunks(myaudio,chunk_length_ms)

    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk 
    for i, audio_chunk in enumerate(chunks, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
#                 print(chunk_filename, ":", text)
                w.append(text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [21]:
path = "C:\\Users\\dobar\\Desktop\\2nd Sem\\Deep Learning\\Project\\Final\\test.wav"
final_text = get_large_audio_transcription(path)

In [22]:
print(final_text)

I work with the banks of mathematicians for separation computer sciences london visit around and think. About the future of machine intelligence mahadev things that some of these things are science fiction name for all their crazy but i like this am ok let's look at the modern human condition this is the normal weight for 6. Pratham if it were actually recently arrived at this final the human species like if a world that was created refer script in one year ago the human species than would be 10 minutes all the industrial era started 2 seconds ago. Another way to look at age if you think of world gdp over the last ten thousand years i've actually taken the trouble is it passes for you in a graph it looks like this it take for normal condition i7 2600. What is the cause of this current animal some people with its technology its true and technology has accumulated to human history and right now technology advances that is the proximate cause that's why we are currently sold reproductive.

# Doing Abstractive Summarization

In [53]:
# Load tokenizer
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-wikihow")

In [54]:
# Load model
modelAbs = PegasusForConditionalGeneration.from_pretrained("google/pegasus-wikihow")

In [55]:
# Return the summary of the input Text
def GenerateSummary(text, x = 0.25):

    listTemp = []
    main_listText = []
    
    inputList = list(text.split('.'))
    l = len(inputList)
    chunk_size = math.ceil(l * x)      
    i = 0
    j = chunk_size - 1
    iterations = l / (chunk_size)

    while i < l:
        listTemp.append(inputList[i])
        if len(listTemp) >= chunk_size:
            main_listText.append('.'.join(listTemp))
            listTemp = []
        i += 1
    if len(listTemp) != 0:
        main_listText.append('.'.join(listTemp))
    
    # Create tokens - number representation of our text
    token = tokenizer(main_listText, truncation = True, padding = "longest", return_tensors = "pt")
    # Summarize
    summary = modelAbs.generate(**token)
    i = 0
    summaryText = ""
    while(i < len(summary)):
        summaryText += tokenizer.decode(summary[i])
        i += 1
    return summaryText

In [10]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

## For WikiHow Data

In [11]:
df = pd.read_csv("wikihowAll.csv")

In [30]:
df.shape

(215365, 3)

In [56]:
body = df['text'][69420]

In [58]:
body

" If you are as young as 14, some companies will not hire you. Get a job (or at least a paper route).;\n, Example: Instead of going on a date to dinner and a movie, go for a stroll through the park and have a picnic. (much more romantic anyway)\n\n, Then you won't be pressured to go to the mall or the movies and spend money with them.\n\n, Tell them that you would be much more encouraged to study harder and put more effort into your work if there was a cash reward. Tell them that it is your future, but you are just a kid. Kids need to work hard at school and have fun. If you had a cash reward for good grades, then you could both be happy.\n\n, Seriously. But write down where you hid it. Or tell someone. Then only allow yourself a certain amount of money a month.\n\n, Give yourself more time to save the money.\n\n, Whatever you are good at you can get paid for. You can tutor kids in a subject area you're especially good at: mow lawns, babysit, clean houses, run errands, anything.\n\n, C

In [57]:
x = 0.2
sum_Text = GenerateSummary(body, x)

In [59]:
sum_Text

"Get a job.<n>Find something fun to do with your date.<n>Tell your date that you need to work hard at school to get good grades.<n>Tell your date that you want a cash reward for good grades.<n>Have fun with your date.Find out how much money you have.<n>Don't buy anything you don't need.<n>Get a job.<n>Ask your parents for a check.Don't buy things you don't need.<n>,<n>Don't buy things you don't need.<n>Don't buy things that are going to wear out.<n>Look around your house for loose change.<n>Shop at discount stores, such as Walmart and Target.<n>Use your gift cards.Use gift cards.<n>Take your change to the ATM.<n><n>,<n>Keep your change in a piggy bank.<n>Take your change to the Coin Master.Convert your change to dollar bills.Go to a bank and ask for some paper coin rolls.Count all the change by hand.<n>Put all extra money in a piggy bank.Go to a bank and ask for some paper coin rolls."

In [12]:
R1 = []
RL = []
for i in range(20):
    body = df['text'][i]
    result = GenerateSummary(body)
    score = scorer.score(df['headline'][i],result)
    r1 = score['rouge1'].recall
    rl = score['rougeL'].recall
    R1.append(r1)
    RL.append(rl)

In [14]:
print(R1)

[0.4117647058823529, 0.553030303030303, 0.6666666666666666, 0.5737704918032787, 0.5, 0.6825396825396826, 0.6923076923076923, 0.6190476190476191, 0.48148148148148145, 0.6521739130434783, 0.39473684210526316, 0.8333333333333334, 0.5789473684210527, 0.3129251700680272, 0.49122807017543857, 0.3956043956043956, 0.7391304347826086, 0.6521739130434783, 0.6666666666666666, 0.6]


In [15]:
print(RL)

[0.3058823529411765, 0.3333333333333333, 0.48717948717948717, 0.3442622950819672, 0.26666666666666666, 0.3968253968253968, 0.5384615384615384, 0.5714285714285714, 0.2716049382716049, 0.5362318840579711, 0.21052631578947367, 0.625, 0.47368421052631576, 0.1564625850340136, 0.34210526315789475, 0.1978021978021978, 0.5, 0.5217391304347826, 0.5, 0.52]


In [46]:
R1 = [0.4117647058823529, 0.553030303030303, 0.6666666666666666, 0.5737704918032787, 0.5, 0.6825396825396826, 0.6923076923076923, 0.6190476190476191, 0.48148148148148145, 0.6521739130434783, 0.39473684210526316, 0.8333333333333334, 0.5789473684210527, 0.3129251700680272, 0.49122807017543857, 0.3956043956043956, 0.7391304347826086, 0.6521739130434783, 0.6666666666666666, 0.6]
RL = [0.3058823529411765, 0.3333333333333333, 0.48717948717948717, 0.3442622950819672, 0.26666666666666666, 0.3968253968253968, 0.5384615384615384, 0.5714285714285714, 0.2716049382716049, 0.5362318840579711, 0.21052631578947367, 0.625, 0.47368421052631576, 0.1564625850340136, 0.34210526315789475, 0.1978021978021978, 0.5, 0.5217391304347826, 0.5, 0.52]

## Average ROUGE-L and ROUGE-1 score

In [47]:
sum(RL)/len(RL)

0.40495980834961953

In [48]:
sum(R1)/len(R1)

0.574876437500141

## Abstractive Summary of Video

In [26]:
x = 0.2
sumText = GenerateSummary(final_text, x)

In [27]:
sumText

"Think about the future of machine intelligence mahadev things that some of these things are science name for all their crazy but i like this let's look at the modern human condition like this.Learn to play any computer game console.<n>Feel very hard work and a few moments later change the staff at human verification it like that is whose right fire was performed implication particular when it comes to questions of power.Recognise that there is no such thing as a human being and that there is no such thing as a race of human beings and that there is no such thing as a species of human beings.The goal is to be comforted not just as a metaphor for greed but as an illustration of a half an hour for optimisation process and give it a misconceived of all specified call if a computer start speaking.<n>The goal is to be comforted not just as a metaphor for greed but as an illustration of a half an hour for optimisation process and give it a misconceived of all specified call if a computer sta

# Doing Extractive Summarization

In [29]:
from summarizer import Summarizer
import torch

In [31]:
model = Summarizer('distilbert-base-uncased', hidden=[-1,-2], hidden_concat=True)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

## For WikiHow Data 

In [36]:
df.title[69420]

'How to Save Up a Lot of Money for a Holiday (for Teens)'

In [37]:
print(df['headline'][0])


Keep related supplies in the same area.,
Make an effort to clean a dedicated workspace after every session.,
Place loose supplies in large, clearly visible containers.,
Use clotheslines and clips to hang sketches, photos, and reference material.,
Use every inch of the room for storage, especially vertical space.,
Use chalkboard paint to make space for drafting ideas right on the walls.,
Purchase a label maker to make your organization strategy semi-permanent.,
Make a habit of throwing out old, excess, or useless stuff each month.


In [32]:
body = df['text'][69420]

In [33]:
body

" If you are as young as 14, some companies will not hire you. Get a job (or at least a paper route).;\n, Example: Instead of going on a date to dinner and a movie, go for a stroll through the park and have a picnic. (much more romantic anyway)\n\n, Then you won't be pressured to go to the mall or the movies and spend money with them.\n\n, Tell them that you would be much more encouraged to study harder and put more effort into your work if there was a cash reward. Tell them that it is your future, but you are just a kid. Kids need to work hard at school and have fun. If you had a cash reward for good grades, then you could both be happy.\n\n, Seriously. But write down where you hid it. Or tell someone. Then only allow yourself a certain amount of money a month.\n\n, Give yourself more time to save the money.\n\n, Whatever you are good at you can get paid for. You can tutor kids in a subject area you're especially good at: mow lawns, babysit, clean houses, run errands, anything.\n\n, C

In [34]:
result = model(body)

In [35]:
result

"If you are as young as 14, some companies will not hire you. much more romantic anyway)\n\n, Then you won't be pressured to go to the mall or the movies and spend money with them. If you had a cash reward for good grades, then you could both be happy. , Give yourself more time to save the money. , You may find money still in old wallets or under the bed, or between sofa cushions. Be sure to wash your hands when you are done. ,,, Put all extra money you earn in a piggy bank. You will be surprised at how much it mounts up!"

In [38]:
y = scorer.score(df['headline'][0],result)

In [39]:
y

{'rouge1': Score(precision=0.1523809523809524, recall=0.18823529411764706, fmeasure=0.16842105263157894),
 'rougeL': Score(precision=0.08571428571428572, recall=0.10588235294117647, fmeasure=0.09473684210526316)}

In [40]:
y['rouge1'].recall

0.18823529411764706

In [41]:
R1 = []
RL = []
for i in range(20):
    body = df['text'][i]
    result = model(body)
    score = scorer.score(df['headline'][i],result)
    r1 = score['rouge1'].recall
    rl = score['rougeL'].recall
    R1.append(r1)
    RL.append(rl)

In [42]:
print(R1)

[0.4117647058823529, 0.36363636363636365, 0.38461538461538464, 0.5163934426229508, 0.4, 0.49206349206349204, 0.4230769230769231, 0.47619047619047616, 0.3950617283950617, 0.4782608695652174, 0.15789473684210525, 0.6666666666666666, 0.47368421052631576, 0.16326530612244897, 0.2894736842105263, 0.5934065934065934, 0.43478260869565216, 0.5652173913043478, 0.5, 0.56]


In [43]:
print(RL)

[0.2235294117647059, 0.18181818181818182, 0.23076923076923078, 0.2459016393442623, 0.16666666666666666, 0.25396825396825395, 0.3076923076923077, 0.38095238095238093, 0.19753086419753085, 0.2898550724637681, 0.13157894736842105, 0.4583333333333333, 0.2807017543859649, 0.09523809523809523, 0.15789473684210525, 0.2802197802197802, 0.2826086956521739, 0.391304347826087, 0.26666666666666666, 0.36]


## Average ROUGE-L and ROUGE-1 score

In [44]:
sum(RL)/len(RL)

0.2591615183584959

In [45]:
sum(R1)/len(R1)

0.437272729191144

## Extractive summarization for video 

In [49]:
summaryText = model(final_text)

In [50]:
summaryText

"I work with the banks of mathematicians for separation computer sciences london visit around and think. Another way to look at age if you think of world gdp over the last ten thousand years i've actually taken the trouble is it passes for you in a graph it looks like this it take for normal condition i7 2600. In this century scientist mahila and awaken the power of artificial intelligence in ice cream i can see and enhance its position when they think about what is smart and what is term i sent have in mind a picture but the like this that one and we have listed yet. What we need to do is to issue more abstract not in terms of the word hollywood soon as we need to think of intelligence as an optimisation process at process that there's the future in a particular set of configuration in talend. Call to make semen smile when it will take control of the world and electrons in the facial muscles of humans. What we just shut it off a chimpanzees and resolve. That we have this on the contra

References :
* Jingqing Zhang, Yao Zhao, Mohammad Saleh, and Peter J. Liu. 2020. PEGASUS: pre-training with extracted gap-sentences for abstractive summarization. In Proceedings of the 37th International Conference on Machine Learning (ICML'2020). JMLR.org, Article 1051, 11328–11339.
* https://github.com/dmmiller612/bert-extractive-summarizer
* Mahnaz Koupaee, William Yang Wang. WikiHow: A Large Scale Text Summarization Dataset. 2018. arXiv:1810.09305 [cs.CL]